In [0]:
# service principal for integrating with ADLS and access it's data

spark.conf.set("fs.azure.account.auth.type.hpadlsacc.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.hpadlsacc.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "app-key"))
spark.conf.set("fs.azure.account.oauth2.client.secret.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "service-cred"))
tenant_id = dbutils.secrets.get("hc-secret-scope", "dir-id")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.hpadlsacc.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# Creating dimensional patient table

In [0]:
%sql
--creating the dim_patient table with ADLS as external location

CREATE TABLE IF NOT EXISTS gold.dim_patient (
    patient_key STRING,
    src_patientid STRING,
    firstname STRING,
    lastname STRING,
    middlename STRING,
    ssn STRING,
    phonenumber STRING,
    gender STRING,
    dob DATE,
    address STRING,
    datasource STRING
)
USING DELTA
LOCATION "abfss://gold@hpadlsacc.dfs.core.windows.net/dim_patient";


In [0]:
%sql
-- reseting the table for each run
truncate table gold.dim_patient;

In [0]:
%sql
-- loading the data from the silver patient table
INSERT INTO gold.dim_patient
SELECT 
    patient_key,
    src_patientid,
    firstname,
    lastname,
    middlename,
    ssn,
    phonenumber,
    gender,
    dob,
    address,
    datasource
FROM silver.patients
WHERE is_current = true AND is_quarantined = false;


In [0]:
%sql
-- getting some record
select * from gold.dim_patient;